# Deep Text Learning

### With Machine Learning techniques in deep learning, classification, and regression, this notebook will demonstrate how to predict a blogger's gender and age with high accuracy based on his or her blog posts

In [3]:
import os
import graphlab as gl
from bs4 import  BeautifulSoup

In [2]:
import graphlab as gl